#  **<center><span style="color:red">Projet Python DU Data Analytics Sorbonne</span></center>**
## **by Joris Salmon**

### Importation des librairies

In [1]:
import os
import pandas as pd
import numpy as np

## **<span style="color:red">INPUT</span>**

### **Ensemble de données sur les transactions des différents exchanges**

À notre disposition se trouve un ensemble de jeux de données répertoriant toutes les transactions effectuées sur les divers exchanges.

1. <span style="color:green">**Lecture des fichiers CSV et stockage :**</span>
   La première étape consiste à lire tous les fichiers CSV et à les stocker.

2. <span style="color:green">**Renommage des colonnes pour garantir l'unicité :**</span>
   Ensuite, il sera nécessaire de renommer chaque colonne des fichiers, notamment les prix, afin de les rendre uniques lors de la fusion ultérieure.

3. <span style="color:green">**Traitement des valeurs manquantes :**</span>
   La troisième étape implique le traitement des valeurs manquantes. Cela se fera en supprimant les lignes qui ne possèdent pas de date et en imputant la moyenne des valeurs pour le prix.

4. <span style="color:green">**Retraitement des dates pour uniformiser le format :**</span>
   Enfin, la quatrième et dernière étape concernera le retraitement des dates pour les convertir en un format standard, assurant ainsi l'uniformité des données. (Nous regrouperons les transactions s'étant effectué dans la même seconde en effectuant la somme du volume et la moyenne du price)

<span style="color:orange">**NB :** Nous profitons de cette étape pour effectuer ces tâches puisque nous parcourons chaque fichier pour les stocker, ce qui optimise le temps d'exécution.</span>


In [2]:
path=os.getcwd()
data_path = os.path.join(path, "input")

def read_all_csv_formate(directory):
    all_dataframes = []
    all_names=[]

    # On parcourt tous les fichiers du répertoire, et on stocke les csv
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_name_without_extension = os.path.splitext(filename)[0] #On prend le nom du fichier sans extension
            all_names.append(file_name_without_extension)
            
            filepath = os.path.join(directory, filename)
            
            # Lire le fichier CSV et stocker le dataframe
            df = pd.read_csv(filepath,sep=",")


            # On supprime les lignes sans dates ou sans volume -> pas d'intérêt puisque l'on ne sait pas quand cela s'est produit et qu'on ne peut pas imputer un volume
            df.dropna(subset=['timestamp','amount'], inplace=True) #Il n'y a pas de lignes de supprimé


            # On remplace les valeurs manquantes par la moyenne pour le prix -> cela évite la reproduction de valeurs aberrantes en prenant la valeur précédente et cela permet d'avoir un impact moindre
            price_mean = df['price'].mean()
            df["price"] = df["price"].fillna(price_mean)

            # On crée dès cette étape une colonne où l'on multiplie le price*volume
            df["Pi*Qj"]=df["price"]*df["amount"]

            
            #On uniformise les dates
            df['timestamp'] = df['timestamp'].str.slice(0, 19)# On ne prend pas en compte les millisecondes, ce qui fait que l'on uniforme les dates entre l'ensemble des dfs
            df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y-%m-%d %H:%M:%S")
            df = df.groupby(pd.Grouper(key='timestamp')).agg({'amount': 'sum', 'price': 'mean', 'Pi*Qj': 'sum'}) # On regroupe les différentes lignes qui étaient en millisecondes pour les regrouper par seconde en faisant une moyenne des prices des transactions et la somme du volume
            
            df.rename(columns={ "Pi*Qj": f'Pi*Qj_{file_name_without_extension}', # On rajoute le nom de l'exchange pour chaque colonne, pas pour la date puisque c'est avec cette variable que nous allons merge les dfs
                    "price" : f'Price_{file_name_without_extension}',
                    'amount': f'amount_{file_name_without_extension}'}, inplace=True)

            all_dataframes.append(df)

    return all_dataframes,all_names

exchanges,names=read_all_csv_formate(data_path) #On stocke tous les dfs et les noms des exchanges
print(exchanges)

[                     amount_bfly  Price_bfly    Pi*Qj_bfly
timestamp                                                 
2021-02-24 23:59:33     0.005400    49712.15    268.445610
2021-02-24 23:44:16     0.077000    49603.44   3819.464880
2021-02-24 22:32:39     0.120000    48864.19   5863.702800
2021-02-24 22:15:19     0.120000    48252.47   5790.296400
2021-02-24 22:15:17     0.120000    48247.38   5789.685600
...                          ...         ...           ...
2021-02-24 00:14:48     0.407834    48263.33  19683.422101
2021-02-24 00:11:03     0.000245    48574.05     11.920072
2021-02-24 00:11:02     0.001000    48574.05     48.574050
2021-02-24 00:05:08     0.010000    48683.05    486.830500
2021-02-24 00:02:59     0.120000    48929.54   5871.544800

[184 rows x 3 columns],                      amount_bfnx    Price_bfnx   Pi*Qj_bfnx
timestamp                                                  
2021-02-24 23:59:58     0.015000  49714.500000   745.725000
2021-02-24 23:59:51     0.0

## **<span style="color:red">MANIPULATION DE DONNEES</span>**

### **<span style="color:green">Maintenant que nos données sont propres, nous allons fusionner l'ensemble des DataFrames contenant les transactions des différents exchanges.</span>**


In [3]:
merged_df=exchanges[0]
for df in exchanges[1:]:#On va joindre tous  les DataFrames ensemble en fonction de la date
    merged_df = pd.merge(merged_df, df, on='timestamp', how='outer')
print(merged_df)

                     amount_bfly  Price_bfly  Pi*Qj_bfly  amount_bfnx  \
timestamp                                                               
2021-02-24 00:00:00          NaN         NaN         NaN      0.02383   
2021-02-24 00:00:01          NaN         NaN         NaN          NaN   
2021-02-24 00:00:02          NaN         NaN         NaN      0.00500   
2021-02-24 00:00:03          NaN         NaN         NaN          NaN   
2021-02-24 00:00:04          NaN         NaN         NaN          NaN   
...                          ...         ...         ...          ...   
2021-02-24 23:59:55          NaN         NaN         NaN          NaN   
2021-02-24 23:59:56          NaN         NaN         NaN          NaN   
2021-02-24 23:59:57          NaN         NaN         NaN          NaN   
2021-02-24 23:59:58          NaN         NaN         NaN      0.01500   
2021-02-24 23:59:59          NaN         NaN         NaN          NaN   

                       Price_bfnx   Pi*Qj_bfnx  am

## **<span style="color:red">CALCUL</span>**

**NB:** <span style="color:green">On obtient un tableau à 33 colonnes et 85000 lignes en sachant qu'une journée est composée d'environ 86000 secondes.</span>

### <span style="color:green">**On calcule la moyenne des prix par heure pour tous les exchanges.**</span>


In [4]:
print(merged_df) #On observe que les exchanges n'enregistrent pas forcément de transactions en même temps, le nombre de transaction est différents

                     amount_bfly  Price_bfly  Pi*Qj_bfly  amount_bfnx  \
timestamp                                                               
2021-02-24 00:00:00          NaN         NaN         NaN      0.02383   
2021-02-24 00:00:01          NaN         NaN         NaN          NaN   
2021-02-24 00:00:02          NaN         NaN         NaN      0.00500   
2021-02-24 00:00:03          NaN         NaN         NaN          NaN   
2021-02-24 00:00:04          NaN         NaN         NaN          NaN   
...                          ...         ...         ...          ...   
2021-02-24 23:59:55          NaN         NaN         NaN          NaN   
2021-02-24 23:59:56          NaN         NaN         NaN          NaN   
2021-02-24 23:59:57          NaN         NaN         NaN          NaN   
2021-02-24 23:59:58          NaN         NaN         NaN      0.01500   
2021-02-24 23:59:59          NaN         NaN         NaN          NaN   

                       Price_bfnx   Pi*Qj_bfnx  am

In [5]:
mean_exchange_one_hour=merged_df.resample("h").agg("sum") #On regroupe par heure en faisant la somme, on a le volume totale vendu et sa valeur pondéré
#Maintenant on peut faire le calcul pour avoir la valeur moyenne d'un volume acheté par heure
def vwap(df,group:str):#calcul de VWAP
    VWAP_name="VWAP_"+group #On iniatialise le nom des colonnes
    PiQj_name="Pi*Qj_"+group
    Amount_name="amount_"+group
    df[VWAP_name] = round(df[PiQj_name] / df[Amount_name],2)
    return df


for name in names:#On le fait pour tous les exchanges
    mean_exchange_one_hour=vwap(mean_exchange_one_hour,name)

print(mean_exchange_one_hour)

                     amount_bfly    Price_bfly     Pi*Qj_bfly  amount_bfnx  \
timestamp                                                                    
2021-02-24 00:00:00     1.957433  1.099477e+06   93970.450429   626.714239   
2021-02-24 01:00:00     1.036138  3.962681e+05   51400.660247   899.689398   
2021-02-24 02:00:00     1.765079  5.528057e+05   88601.110515   756.814445   
2021-02-24 03:00:00     0.250000  5.094800e+04   12737.000000   503.342279   
2021-02-24 04:00:00     0.770395  1.017600e+05   39207.574652   565.681941   
2021-02-24 05:00:00     0.000000  0.000000e+00       0.000000   502.864453   
2021-02-24 06:00:00     1.570401  7.494308e+05   78486.302646   899.859603   
2021-02-24 07:00:00     0.032677  4.968010e+04    1623.412525   267.211745   
2021-02-24 08:00:00     0.788532  2.014381e+05   39540.309119   739.579049   
2021-02-24 09:00:00     0.350000  2.035446e+05   17809.408300   736.825144   
2021-02-24 10:00:00     0.138200  1.010304e+05    6984.158402   

### **<span style="color:green">NB : c'est une première façon de faire, à la suite nous verrons comment procéder autrement avec apply</span>**

### Maintenant on calcul la moyenne par heure pour chaque exchange

In [6]:
VWAP_columns= [col for col in mean_exchange_one_hour.columns if col.startswith("VWAP")]
mean_exchange_one_hour_VWAP=mean_exchange_one_hour[VWAP_columns]
mean_exchange_one_hour_VWAP["Global_mean"] = mean_exchange_one_hour_VWAP.mean(axis=1)
mean_exchange_one_hour_VWAP=mean_exchange_one_hour_VWAP.fillna("NO TRADES")
print(mean_exchange_one_hour_VWAP)

                     VWAP_bfly  VWAP_bfnx  VWAP_bnus  VWAP_btrx  VWAP_cbse  \
timestamp                                                                    
2021-02-24 00:00:00   48006.99   47778.97   47846.76   47861.25   47945.34   
2021-02-24 01:00:00   49607.92   49441.59   49196.19   49343.27   49317.20   
2021-02-24 02:00:00   50196.68   50237.01   50200.41   50264.13   50248.89   
2021-02-24 03:00:00    50948.0   50608.44   50523.29   50528.68   50498.14   
2021-02-24 04:00:00   50892.81   50889.74   50900.80   50914.28   50942.65   
2021-02-24 05:00:00  NO TRADES   50344.15   50383.99   50537.36   50513.99   
2021-02-24 06:00:00   49978.51   49713.51   50046.49   50033.77   49982.47   
2021-02-24 07:00:00    49680.1   49899.14   49861.58   50032.96   49959.62   
2021-02-24 08:00:00   50144.21   50629.72   50559.89   50587.22   50541.09   
2021-02-24 09:00:00   50884.02   50887.44   50842.08   50947.53   50901.00   
2021-02-24 10:00:00    50536.6   50480.38   50515.87   50665.39 

C:\Users\ingemedia\AppData\Local\Temp\ipykernel_23488\4126484045.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mean_exchange_one_hour_VWAP["Global_mean"] = mean_exchange_one_hour_VWAP.mean(axis=1)


### Maintenant on exporte le résultat en fichier csv

In [7]:
mean_exchange_one_hour_VWAP.to_csv("./output/VWAP/VWAP_exchanges_one_hour.csv",sep=",")

## **<center><span style="color:red">ALLER PLUS LOIN</span></center>**

### On va reprendre le df merged_df contenant tous les dfs et on va appliquer nos fonctions dessus

In [8]:
print(merged_df)

                     amount_bfly  Price_bfly  Pi*Qj_bfly  amount_bfnx  \
timestamp                                                               
2021-02-24 00:00:00          NaN         NaN         NaN      0.02383   
2021-02-24 00:00:01          NaN         NaN         NaN          NaN   
2021-02-24 00:00:02          NaN         NaN         NaN      0.00500   
2021-02-24 00:00:03          NaN         NaN         NaN          NaN   
2021-02-24 00:00:04          NaN         NaN         NaN          NaN   
...                          ...         ...         ...          ...   
2021-02-24 23:59:55          NaN         NaN         NaN          NaN   
2021-02-24 23:59:56          NaN         NaN         NaN          NaN   
2021-02-24 23:59:57          NaN         NaN         NaN          NaN   
2021-02-24 23:59:58          NaN         NaN         NaN      0.01500   
2021-02-24 23:59:59          NaN         NaN         NaN          NaN   

                       Price_bfnx   Pi*Qj_bfnx  am

### Ici on va définir l'ensemble des calculs demandés à l'aide de fonctions

In [9]:
def vwmp(df, group):
    df=pd.DataFrame(df)
    Price_name = "Price_" + group
    PiQj_name = "Pi*Qj_" + group
    Amount_name = "amount_" + group

        
    # On supprime les lignes contenant des valeurs NaN
    df = df.dropna(subset=[Price_name, Amount_name])
    
    # On trie le prix
    df = df.sort_values(by=Price_name)
    
    # les volumes cumulés
    cumsum_volumes = df[Amount_name].cumsum()

    # on trouve l'index où le volume cumulé dépasse la moitié du volume total
    if not cumsum_volumes.empty:
        total_volume = cumsum_volumes.iloc[-1]
        median_index = cumsum_volumes.searchsorted(total_volume / 2)
    
        # Si le nombre d'éléments est impair
        if total_volume % 2 != 0:
            median_price = df.iloc[median_index][Price_name]
        # Si le nombre d'éléments est pair
        else:
            median_price = (df.iloc[median_index][Price_name] + df.iloc[median_index + 1][Price_name]) / 2
    else:
        return None
    return round(median_price,2)

def ecart_type(df, group):
    df = pd.DataFrame(df)
    Price_name = "Price_" + group
    PiQj_name = "Pi*Qj_" + group
    Amount_name = "amount_" + group

    # On supprime les lignes contenant des valeurs NaN
    df = df.dropna(subset=[Price_name, Amount_name])

    # On calcule l'écart-type
    if not df.empty:
        std_deviation = round(np.std(df[Price_name]),2)
    else:
        std_deviation = None
    return std_deviation

def OHLC(df, group):
    df = pd.DataFrame(df)
    Price_name = "Price_" + group
    PiQj_name = "Pi*Qj_" + group
    Amount_name = "amount_" + group

    # On supprime les lignes contenant des valeurs NaN
    df = df.dropna(subset=[Price_name, Amount_name])

    # Calculer les valeurs OHLC (Open, High, Low, Close)
    if not df.empty:
        open_price = df[Price_name].iloc[0]
        high_price = df[Price_name].max()
        low_price = df[Price_name].min()
        close_price = df[Price_name].iloc[-1]
        return {
            "Open": round(open_price, 2),
            "High": round(high_price, 2),
            "Low": round(low_price, 2),
            "Close": round(close_price, 2)
        }
    else:
        return None

def calculate(indicateur=vwmp, frequence="h", names=names, merged_df=merged_df,global_mean=True):
    mean_exchange_df = pd.DataFrame()
    for name in names:
        result = merged_df.resample(frequence).apply(lambda x: indicateur(x, name))
        name_column = str(indicateur.__name__) + "_"+name
        mean_exchange_df[name_column] = result
    if global_mean==True:
        mean_exchange_df["Global_mean"]=mean_exchange_df.mean(axis=1)
    mean_exchange_df=mean_exchange_df.fillna("NO TRADES")
    mean_exchange_df.to_csv(path+"\\output\\"+str(indicateur.__name__)+"\\"+str(indicateur.__name__)+"_"+frequence+".csv", sep=",")
    return mean_exchange_df


#### **VWMP** à différentes fréquences

In [10]:
calculate(indicateur=vwmp, frequence="h", names=names, merged_df=merged_df)

,vwmp_bfly,vwmp_bfnx,vwmp_bnus,vwmp_btrx,vwmp_cbse,vwmp_gmni,vwmp_itbi,vwmp_krkn,vwmp_lmax,vwmp_okcn,vwmp_stmp,Global_mean
timestamp,,,,,,,,,,,,
2021-02-24 00:00:00,48128.62,47621.66,47714.45,47749.53,47784.69,47780.84,47682.67,47658.38,47730.76,48000.00,47791.27,47785.715455
2021-02-24 01:00:00,49818.51,49548.06,49120.79,49269.87,49276.69,49558.04,49700.00,49075.09,49192.06,49658.66,49021.74,49385.410000
2021-02-24 02:00:00,50200.0,50271.50,50224.79,50300.00,50278.29,50294.70,50267.42,50207.98,50276.64,50257.69,50304.48,50262.135455
2021-02-24 03:00:00,50948.0,50674.00,50450.50,50488.86,50422.36,50503.49,50433.00,50450.10,50382.50,50381.43,50405.71,50503.631818
2021-02-24 04:00:00,51019.92,50909.46,50936.60,50916.64,50979.60,50979.12,51012.00,50940.79,50838.50,51111.70,51001.28,50967.782727
2021-02-24 05:00:00,NO TRADES,50282.00,50328.65,50500.00,50437.94,50427.81,50363.38,50361.70,50359.01,50372.03,50389.19,50382.171000
2021-02-24 06:00:00,50018.9,49635.00,50000.00,49964.33,49898.53,49857.01,49779.08,49835.99,50026.17,49893.06,49875.34,49889.400909
2021-02-24 07:00:00,49680.1,49915.39,49889.21,50042.35,49966.79,50064.46,49934.25,50025.00,50047.17,50084.43,50002.00,49968.286364
2021-02-24 08:00:00,49890.34,50658.75,50635.10,50692.39,50609.71,50514.06,50642.50,50635.27,50636.50,50538.89,50642.75,50554.205455


In [11]:
calculate(indicateur=vwmp, frequence="5Min", names=names, merged_df=merged_df)

,vwmp_bfly,vwmp_bfnx,vwmp_bnus,vwmp_btrx,vwmp_cbse,vwmp_gmni,vwmp_itbi,vwmp_krkn,vwmp_lmax,vwmp_okcn,vwmp_stmp,Global_mean
timestamp,,,,,,,,,,,,
2021-02-24 00:00:00,48929.54,48894.22,48916.43,48879.88,48901.10,48962.61,48979.67,48899.9,48901.35,48927.54,48963.30,48923.230909
2021-02-24 00:05:00,48683.05,48568.62,48607.86,48577.71,48629.96,48701.00,48627.50,NO TRADES,48590.12,48603.23,48606.23,48619.528000
2021-02-24 00:10:00,48263.33,48287.71,48450.00,48380.97,48462.34,48469.77,48425.00,48495.22,48532.5,48376.83,48578.82,48429.317273
2021-02-24 00:15:00,47743.62,47653.50,47848.30,47895.06,47787.19,47900.56,47933.19,47861.98,47826.69,47609.33,47988.19,47822.510000
2021-02-24 00:20:00,47388.0,47238.90,47265.92,47346.22,47335.76,47256.16,47233.00,47293.64,47250.59,47253.61,47209.10,47279.172727
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24 23:35:00,NO TRADES,49612.52,49560.47,49713.55,49612.90,49618.80,49594.65,49621.0,49638.33,49700.00,49631.12,49630.334000
2021-02-24 23:40:00,49603.44,49537.00,49584.62,49615.51,49601.91,49604.50,49601.38,49597.32,49583.75,49595.74,49597.46,49592.966364
2021-02-24 23:45:00,NO TRADES,49662.98,49550.04,49712.18,49702.65,49730.78,49702.00,49637.92,49708.75,49698.90,49700.00,49680.620000


In [12]:
calculate(indicateur=vwmp, frequence="30Min", names=names, merged_df=merged_df)

,vwmp_bfly,vwmp_bfnx,vwmp_bnus,vwmp_btrx,vwmp_cbse,vwmp_gmni,vwmp_itbi,vwmp_krkn,vwmp_lmax,vwmp_okcn,vwmp_stmp,Global_mean
timestamp,,,,,,,,,,,,
2021-02-24 00:00:00,48263.33,47595.97,47841.56,47679.13,47848.33,48071.06,47687.00,47372.20,47758.48,47795.03,47995.85,47809.812727
2021-02-24 00:30:00,48128.62,47703.33,47708.58,47771.01,47747.85,47732.58,47680.00,47735.85,47718.47,48000.00,47771.02,47790.664545
2021-02-24 01:00:00,48934.14,49000.04,48847.87,48965.34,48944.60,48854.80,49016.25,48834.94,48807.13,48874.96,49000.00,48916.370000
2021-02-24 01:30:00,49818.51,49728.76,49680.90,49791.47,49710.32,49832.02,49700.11,49625.31,49728.50,49855.89,49716.76,49744.413636
2021-02-24 02:00:00,50200.0,50243.94,50145.01,50237.66,50180.26,50216.75,50289.75,50166.65,50183.50,50144.46,50219.44,50202.492727
2021-02-24 02:30:00,NO TRADES,50300.00,50301.89,50313.68,50326.61,50330.97,50267.42,50311.10,50311.33,50294.39,50338.98,50309.637000
2021-02-24 03:00:00,NO TRADES,50306.19,50348.70,50488.86,50374.49,50452.08,50324.62,50380.33,50370.75,50352.96,50389.09,50378.807000
2021-02-24 03:30:00,50948.0,50800.00,50777.87,50777.00,50554.25,50526.83,50649.00,50724.10,50397.15,50793.51,50672.07,50692.707273
2021-02-24 04:00:00,51019.92,51058.00,51030.03,51105.12,51102.73,51072.62,51094.62,51075.90,51117.50,51218.46,51117.17,51092.006364


#### **Ecart-type** à différentes fréquences

In [13]:
calculate(indicateur=ecart_type, frequence="h", names=names, merged_df=merged_df)

,ecart_type_bfly,ecart_type_bfnx,ecart_type_bnus,ecart_type_btrx,ecart_type_cbse,ecart_type_gmni,ecart_type_itbi,ecart_type_krkn,ecart_type_lmax,ecart_type_okcn,ecart_type_stmp,Global_mean
timestamp,,,,,,,,,,,,
2021-02-24 00:00:00,527.54,538.65,525.35,521.48,499.21,525.57,503.50,413.11,569.35,519.69,528.98,515.675455
2021-02-24 01:00:00,467.96,444.11,444.46,449.12,436.92,451.49,435.73,455.10,497.34,423.79,431.39,448.855455
2021-02-24 02:00:00,150.16,166.85,174.31,169.58,168.37,170.31,167.19,180.58,175.06,165.87,168.58,168.805455
2021-02-24 03:00:00,0.0,231.72,206.60,210.44,208.09,213.24,206.47,226.83,196.79,184.45,204.48,189.919091
2021-02-24 04:00:00,139.92,230.64,230.52,237.58,234.85,232.64,228.29,236.87,257.57,231.14,236.68,226.972727
2021-02-24 05:00:00,NO TRADES,255.73,255.60,261.69,243.38,252.62,248.55,249.91,271.19,262.89,228.40,252.996000
2021-02-24 06:00:00,116.1,298.53,307.87,317.32,296.84,288.93,306.86,298.24,296.28,280.10,282.89,280.905455
2021-02-24 07:00:00,0.0,196.34,184.58,182.37,182.00,179.66,184.57,186.08,175.05,176.15,182.84,166.330909
2021-02-24 08:00:00,300.0,225.32,207.33,224.17,220.81,223.25,222.69,211.07,213.28,233.46,214.87,226.931818


In [14]:
calculate(indicateur=ecart_type, frequence="5Min", names=names, merged_df=merged_df)

,ecart_type_bfly,ecart_type_bfnx,ecart_type_bnus,ecart_type_btrx,ecart_type_cbse,ecart_type_gmni,ecart_type_itbi,ecart_type_krkn,ecart_type_lmax,ecart_type_okcn,ecart_type_stmp,Global_mean
timestamp,,,,,,,,,,,,
2021-02-24 00:00:00,0.0,143.55,137.16,123.44,143.46,139.22,138.50,0.1,122.56,132.18,143.63,111.254545
2021-02-24 00:05:00,0.0,60.52,46.70,53.86,52.32,55.17,55.68,NO TRADES,56.17,50.62,54.11,48.515000
2021-02-24 00:10:00,146.47,118.08,108.99,108.50,103.10,122.12,108.01,128.4,141.63,100.97,110.97,117.930909
2021-02-24 00:15:00,84.37,137.92,134.48,137.83,139.60,135.44,143.75,145.64,141.1,141.87,144.55,135.140909
2021-02-24 00:20:00,143.36,228.04,229.94,218.97,228.90,219.18,229.11,230.59,213.22,218.32,215.37,215.909091
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24 23:35:00,NO TRADES,71.90,72.95,71.93,77.07,70.69,76.01,79.27,57.3,96.06,78.77,75.195000
2021-02-24 23:40:00,0.0,38.44,40.57,30.38,41.16,43.67,34.41,38.34,36.07,31.02,42.99,34.277273
2021-02-24 23:45:00,NO TRADES,41.54,45.09,39.19,41.53,37.24,33.82,45.23,38.04,48.81,50.78,42.127000


In [15]:
calculate(indicateur=ecart_type, frequence="30Min", names=names, merged_df=merged_df)

,ecart_type_bfly,ecart_type_bfnx,ecart_type_bnus,ecart_type_btrx,ecart_type_cbse,ecart_type_gmni,ecart_type_itbi,ecart_type_krkn,ecart_type_lmax,ecart_type_okcn,ecart_type_stmp,Global_mean
timestamp,,,,,,,,,,,,
2021-02-24 00:00:00,584.5,651.25,642.82,606.80,637.49,651.95,644.79,487.01,682.91,611.70,642.03,622.113636
2021-02-24 00:30:00,307.7,293.91,280.53,274.42,263.01,241.79,263.09,299.98,269.82,273.31,274.17,276.520909
2021-02-24 01:00:00,67.72,235.74,229.55,225.31,226.94,241.67,240.51,227.47,248.54,225.59,227.59,217.875455
2021-02-24 01:30:00,304.41,358.37,336.28,358.01,357.64,355.78,346.22,358.67,355.47,336.92,356.08,347.622727
2021-02-24 02:00:00,150.16,189.70,204.65,186.98,197.36,195.20,191.76,208.02,214.35,203.39,192.95,194.047273
2021-02-24 02:30:00,NO TRADES,102.80,97.21,104.16,98.66,96.94,96.77,98.60,101.10,100.24,93.32,98.980000
2021-02-24 03:00:00,NO TRADES,95.92,92.77,87.01,89.85,97.34,93.63,93.58,88.08,85.06,86.70,90.994000
2021-02-24 03:30:00,0.0,234.63,227.50,222.19,239.62,247.52,238.73,237.62,233.34,235.87,234.67,213.790000
2021-02-24 04:00:00,139.92,153.63,155.37,149.76,153.33,157.60,149.29,146.68,144.83,139.74,168.00,150.740909


#### **OHLC** avec différentes fréquences

In [16]:
calculate(indicateur=OHLC, frequence="h", names=names, merged_df=merged_df,global_mean=False)

,OHLC_bfly,OHLC_bfnx,OHLC_bnus,OHLC_btrx,OHLC_cbse,OHLC_gmni,OHLC_itbi,OHLC_krkn,OHLC_lmax,OHLC_okcn,OHLC_stmp
timestamp,,,,,,,,,,,
2021-02-24 00:00:00,"{'Open': 48929.54, 'High': 48929.54, 'Low': 47...","{'Open': 48871.67, 'High': 49086.5, 'Low': 469...","{'Open': 48875.0, 'High': 49096.01, 'Low': 470...","{'Open': 48906.61, 'High': 49058.07, 'Low': 46...","{'Open': 48900.0, 'High': 49106.63, 'Low': 470...","{'Open': 48923.48, 'High': 49123.53, 'Low': 47...","{'Open': 48919.5, 'High': 49100.0, 'Low': 4702...","{'Open': 48899.6, 'High': 48899.9, 'Low': 4701...","{'Open': 48892.5, 'High': 49080.5, 'Low': 4693...","{'Open': 48913.57, 'High': 49078.28, 'Low': 47...","{'Open': 48927.33, 'High': 49083.52, 'Low': 47..."
2021-02-24 01:00:00,"{'Open': 48934.14, 'High': 50161.64, 'Low': 48...","{'Open': 48454.0, 'High': 50227.5, 'Low': 4841...","{'Open': 48440.07, 'High': 50214.49, 'Low': 48...","{'Open': 48488.0, 'High': 50216.29, 'Low': 484...","{'Open': 48503.31, 'High': 50313.18, 'Low': 48...","{'Open': 48510.28, 'High': 50253.18, 'Low': 48...","{'Open': 48485.25, 'High': 50208.25, 'Low': 48...","{'Open': 48499.0, 'High': 50200.0, 'Low': 4847...","{'Open': 48501.75, 'High': 50232.88, 'Low': 48...","{'Open': 48527.41, 'High': 50201.53, 'Low': 48...","{'Open': 48480.9, 'High': 50232.0, 'Low': 4843..."
2021-02-24 02:00:00,"{'Open': 49994.55, 'High': 50513.79, 'Low': 49...","{'Open': 50183.67, 'High': 50552.41, 'Low': 49...","{'Open': 50193.2, 'High': 50544.09, 'Low': 498...","{'Open': 50198.75, 'High': 50572.8, 'Low': 498...","{'Open': 50251.42, 'High': 50594.54, 'Low': 49...","{'Open': 50257.72, 'High': 50598.32, 'Low': 49...","{'Open': 50193.25, 'High': 50537.62, 'Low': 49...","{'Open': 50211.07, 'High': 50564.5, 'Low': 498...","{'Open': 50181.0, 'High': 50569.0, 'Low': 4980...","{'Open': 50240.11, 'High': 50567.44, 'Low': 49...","{'Open': 50224.22, 'High': 50602.39, 'Low': 49..."
2021-02-24 03:00:00,"{'Open': 50948.0, 'High': 50948.0, 'Low': 5094...","{'Open': 50161.0, 'High': 51000.07, 'Low': 500...","{'Open': 50173.46, 'High': 50976.25, 'Low': 50...","{'Open': 50213.58, 'High': 50999.99, 'Low': 50...","{'Open': 50220.01, 'High': 50999.3, 'Low': 500...","{'Open': 50200.0, 'High': 51000.0, 'Low': 5006...","{'Open': 50166.25, 'High': 50991.25, 'Low': 50...","{'Open': 50202.2, 'High': 50988.66, 'Low': 500...","{'Open': 50171.0, 'High': 50999.5, 'Low': 5007...","{'Open': 50186.42, 'High': 50970.65, 'Low': 50...","{'Open': 50190.12, 'High': 51000.0, 'Low': 500..."
2021-02-24 04:00:00,"{'Open': 51019.92, 'High': 51019.92, 'Low': 50...","{'Open': 50901.0, 'High': 51353.0, 'Low': 5034...","{'Open': 50919.02, 'High': 51399.11, 'Low': 50...","{'Open': 51000.0, 'High': 51382.63, 'Low': 504...","{'Open': 50934.55, 'High': 51397.56, 'Low': 50...","{'Open': 50943.71, 'High': 51387.91, 'Low': 50...","{'Open': 50913.25, 'High': 51391.5, 'Low': 503...","{'Open': 50926.6, 'High': 51392.0, 'Low': 5035...","{'Open': 50940.12, 'High': 51409.0, 'Low': 503...","{'Open': 50962.95, 'High': 51379.63, 'Low': 50...","{'Open': 50937.52, 'High': 51391.36, 'Low': 50..."
2021-02-24 05:00:00,NO TRADES,"{'Open': 50611.0, 'High': 51062.75, 'Low': 499...","{'Open': 50656.06, 'High': 51026.61, 'Low': 49...","{'Open': 50676.45, 'High': 51030.52, 'Low': 50...","{'Open': 50655.08, 'High': 51025.09, 'Low': 49...","{'Open': 50713.91, 'High': 51014.44, 'Low': 50...","{'Open': 50656.5, 'High': 51044.5, 'Low': 5001...","{'Open': 50647.5, 'High': 51046.25, 'Low': 500...","{'Open': 50761.3, 'High': 51032.5, 'Low': 5002...","{'Open': 50721.28, 'High': 51044.41, 'Low': 50...","{'Open': 50665.34, 'High': 51039.0, 'Low': 500..."
2021-02-24 06:00:00,"{'Open': 50050.8, 'High': 50050.8, 'Low': 4965...","{'Open': 50167.0, 'High': 50638.67, 'Low': 493...","{'Open': 50211.36, 'High': 50678.4, 'Low': 494...","{'Open': 50232.1, 'High': 50699.1, 'Low': 4950...","{'Open': 50234.41, 'High': 50689.14, 'Low': 49...","{'Open': 50253.37, 'High': 50702.67, 'Low': 49...","{'Open': 50248.25, 'High'

In [17]:
calculate(indicateur=OHLC, frequence="5Min", names=names, merged_df=merged_df,global_mean=False)

,OHLC_bfly,OHLC_bfnx,OHLC_bnus,OHLC_btrx,OHLC_cbse,OHLC_gmni,OHLC_itbi,OHLC_krkn,OHLC_lmax,OHLC_okcn,OHLC_stmp
timestamp,,,,,,,,,,,
2021-02-24 00:00:00,"{'Open': 48929.54, 'High': 48929.54, 'Low': 48...","{'Open': 48871.67, 'High': 49086.5, 'Low': 485...","{'Open': 48875.0, 'High': 49096.01, 'Low': 485...","{'Open': 48906.61, 'High': 49058.07, 'Low': 48...","{'Open': 48900.0, 'High': 49106.63, 'Low': 485...","{'Open': 48923.48, 'High': 49123.53, 'Low': 48...","{'Open': 48919.5, 'High': 49100.0, 'Low': 4858...","{'Open': 48899.6, 'High': 48899.9, 'Low': 4889...","{'Open': 48892.5, 'High': 49080.5, 'Low': 4855...","{'Open': 48913.57, 'High': 49078.28, 'Low': 48...","{'Open': 48927.33, 'High': 49083.52, 'Low': 48..."
2021-02-24 00:05:00,"{'Open': 48683.05, 'High': 48683.05, 'Low': 48...","{'Open': 48603.0, 'High': 48707.5, 'Low': 4847...","{'Open': 48638.3, 'High': 48709.53, 'Low': 485...","{'Open': 48686.85, 'High': 48707.82, 'Low': 48...","{'Open': 48603.25, 'High': 48725.83, 'Low': 48...","{'Open': 48649.08, 'High': 48747.55, 'Low': 48...","{'Open': 48650.25, 'High': 48775.75, 'Low': 48...",NO TRADES,"{'Open': 48694.0, 'High': 48761.0, 'Low': 4850...","{'Open': 48632.03, 'High': 48722.28, 'Low': 48...","{'Open': 48742.39, 'High': 48742.39, 'Low': 48..."
2021-02-24 00:10:00,"{'Open': 48574.05, 'High': 48574.05, 'Low': 48...","{'Open': 48576.0, 'High': 48578.5, 'Low': 4808...","{'Open': 48593.9, 'High': 48607.31, 'Low': 481...","{'Open': 48588.44, 'High': 48620.09, 'Low': 48...","{'Open': 48627.6, 'High': 48635.14, 'Low': 481...","{'Open': 48630.73, 'High': 48630.73, 'Low': 48...","{'Open': 48592.75, 'High': 48629.62, 'Low': 48...","{'Open': 48899.57, 'High': 48899.57, 'Low': 48...","{'Open': 48592.0, 'High': 48608.0, 'Low': 4812...","{'Open': 48593.07, 'High': 48631.26, 'Low': 48...","{'Open': 48597.69, 'High': 48661.95, 'Low': 48..."
2021-02-24 00:15:00,"{'Open': 47873.53, 'High': 47873.53, 'Low': 47...","{'Open': 48090.7, 'High': 48090.7, 'Low': 4755...","{'Open': 48104.76, 'High': 48104.76, 'Low': 47...","{'Open': 48154.66, 'High': 48154.66, 'Low': 47...","{'Open': 48138.8, 'High': 48138.8, 'Low': 4759...","{'Open': 48147.06, 'High': 48147.06, 'Low': 47...","{'Open': 48109.75, 'High': 48109.75, 'Low': 47...","{'Open': 48149.48, 'High': 48149.47, 'Low': 47...","{'Open': 48130.5, 'High': 48130.5, 'Low': 4757...","{'Open': 48098.98, 'High': 48098.98, 'Low': 47...","{'Open': 48130.31, 'High': 48130.31, 'Low': 47..."
2021-02-24 00:20:00,"{'Open': 47500.0, 'High': 47500.0, 'Low': 4708...","{'Open': 47748.0, 'High': 47748.0, 'Low': 4695...","{'Open': 47778.05, 'High': 47778.05, 'Low': 47...","{'Open': 47766.73, 'High': 47766.73, 'Low': 46...","{'Open': 47793.72, 'High': 47793.72, 'Low': 47...","{'Open': 47812.0, 'High': 47811.99, 'Low': 470...","{'Open': 47783.67, 'High': 47783.67, 'Low': 47...","{'Open': 47786.43, 'High': 47786.43, 'Low': 47...","{'Open': 47769.75, 'High': 47769.75, 'Low': 46...","{'Open': 47706.19, 'High': 47748.9, 'Low': 470...","{'Open': 47765.18, 'High': 47765.18, 'Low': 47..."
...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24 23:35:00,NO TRADES,"{'Open': 49393.0, 'High': 49744.0, 'Low': 4939...","{'Open': 49365.54, 'High': 49750.0, 'Low': 493...","{'Open': 49436.22, 'High': 49747.17, 'Low': 49...","{'Open': 49398.17, 'High': 49762.44, 'Low': 49...","{'Open': 49425.51, 'High': 49784.04, 'Low': 49...","{'Open': 49400.25, 'High': 49760.38, 'Low': 49...","{'Open': 49379.15, 'High': 49746.9, 'Low': 493...","{'Open': 49533.5, 'High': 49767.5, 'Low': 4952...","{'Open': 49396.85, 'High': 49786.94, 'Low': 49...","{'Open': 49385.42, 'High': 49774.85, 'Low': 49..."
2021-02-24 23:40:00,"{'Open': 49603.44, 'High': 49603.44, 'Low': 49...","{'Open': 49580.0, 'High': 49616.0, 'Low': 4948...","{'Open': 49597.83, 'High': 49624.91, 'Low': 49...","{'Open': 49596.47, 'High': 49645.94, 'Low': 49...","{'Open': 49590.14, 'High': 49638.99, 'Low': 49...","{'Open': 49629.76, 'High': 49662.04, 'Low': 49...","{'Open': 49587.25, 

In [18]:
calculate(indicateur=OHLC, frequence="30Min", names=names, merged_df=merged_df,global_mean=False)

,OHLC_bfly,OHLC_bfnx,OHLC_bnus,OHLC_btrx,OHLC_cbse,OHLC_gmni,OHLC_itbi,OHLC_krkn,OHLC_lmax,OHLC_okcn,OHLC_stmp
timestamp,,,,,,,,,,,
2021-02-24 00:00:00,"{'Open': 48929.54, 'High': 48929.54, 'Low': 47...","{'Open': 48871.67, 'High': 49086.5, 'Low': 469...","{'Open': 48875.0, 'High': 49096.01, 'Low': 470...","{'Open': 48906.61, 'High': 49058.07, 'Low': 46...","{'Open': 48900.0, 'High': 49106.63, 'Low': 470...","{'Open': 48923.48, 'High': 49123.53, 'Low': 47...","{'Open': 48919.5, 'High': 49100.0, 'Low': 4702...","{'Open': 48899.6, 'High': 48899.9, 'Low': 4701...","{'Open': 48892.5, 'High': 49080.5, 'Low': 4693...","{'Open': 48913.57, 'High': 49078.28, 'Low': 47...","{'Open': 48927.33, 'High': 49083.52, 'Low': 47..."
2021-02-24 00:30:00,"{'Open': 47399.97, 'High': 48191.14, 'Low': 47...","{'Open': 47166.5, 'High': 48494.33, 'Low': 471...","{'Open': 47272.36, 'High': 48482.38, 'Low': 47...","{'Open': 47289.12, 'High': 48488.0, 'Low': 472...","{'Open': 47245.34, 'High': 48515.07, 'Low': 47...","{'Open': 47255.85, 'High': 48513.17, 'Low': 47...","{'Open': 47270.25, 'High': 48485.25, 'Low': 47...","{'Open': 47225.0, 'High': 48499.0, 'Low': 4722...","{'Open': 47248.5, 'High': 48504.0, 'Low': 4724...","{'Open': 47372.7, 'High': 48450.6, 'Low': 4737...","{'Open': 47275.7, 'High': 48513.29, 'Low': 472..."
2021-02-24 01:00:00,"{'Open': 48934.14, 'High': 48934.14, 'Low': 48...","{'Open': 48454.0, 'High': 49355.0, 'Low': 4841...","{'Open': 48440.07, 'High': 49350.59, 'Low': 48...","{'Open': 48488.0, 'High': 49341.07, 'Low': 484...","{'Open': 48503.31, 'High': 49375.0, 'Low': 484...","{'Open': 48510.28, 'High': 49379.87, 'Low': 48...","{'Open': 48485.25, 'High': 49333.0, 'Low': 484...","{'Open': 48499.0, 'High': 49333.0, 'Low': 4847...","{'Open': 48501.75, 'High': 49378.5, 'Low': 484...","{'Open': 48527.41, 'High': 49365.8, 'Low': 484...","{'Open': 48480.9, 'High': 49346.32, 'Low': 484..."
2021-02-24 01:30:00,"{'Open': 49140.16, 'High': 50161.64, 'Low': 49...","{'Open': 49184.5, 'High': 50227.5, 'Low': 4876...","{'Open': 49179.26, 'High': 50214.49, 'Low': 48...","{'Open': 49157.91, 'High': 50216.29, 'Low': 48...","{'Open': 49212.05, 'High': 50313.18, 'Low': 48...","{'Open': 49233.05, 'High': 50253.18, 'Low': 48...","{'Open': 49219.0, 'High': 50208.25, 'Low': 487...","{'Open': 49200.65, 'High': 50200.0, 'Low': 487...","{'Open': 49209.5, 'High': 50232.88, 'Low': 487...","{'Open': 49218.55, 'High': 50201.53, 'Low': 48...","{'Open': 49180.68, 'High': 50232.0, 'Low': 487..."
2021-02-24 02:00:00,"{'Open': 49994.55, 'High': 50513.79, 'Low': 49...","{'Open': 50183.67, 'High': 50552.41, 'Low': 49...","{'Open': 50193.2, 'High': 50544.09, 'Low': 498...","{'Open': 50198.75, 'High': 50572.8, 'Low': 498...","{'Open': 50251.42, 'High': 50594.54, 'Low': 49...","{'Open': 50257.72, 'High': 50598.32, 'Low': 49...","{'Open': 50193.25, 'High': 50537.62, 'Low': 49...","{'Open': 50211.07, 'High': 50564.5, 'Low': 498...","{'Open': 50181.0, 'High': 50569.0, 'Low': 4980...","{'Open': 50240.11, 'High': 50567.44, 'Low': 49...","{'Open': 50224.22, 'High': 50602.39, 'Low': 49..."
2021-02-24 02:30:00,NO TRADES,"{'Open': 50031.33, 'High': 50477.68, 'Low': 50...","{'Open': 50031.42, 'High': 50478.14, 'Low': 50...","{'Open': 50088.59, 'High': 50480.56, 'Low': 50...","{'Open': 50052.04, 'High': 50488.99, 'Low': 50...","{'Open': 50086.54, 'High': 50475.0, 'Low': 500...","{'Open': 50049.88, 'High': 50466.12, 'Low': 50...","{'Open': 50037.3, 'High': 50474.9, 'Low': 5003...","{'Open': 50120.0, 'High': 50485.0, 'Low': 5003...","{'Open': 50118.78, 'High': 50463.98, 'Low': 50...","{'Open': 50049.83, 'High': 50471.57, 'Low': 50..."
2021-02-24 03:00:00,NO TRADES,"{'Open': 50161.0, 'High': 50511.36, 'Low': 500...","{'Open': 50173.46, 'High': 50531.36, 'Low': 50...","{'Open': 50213.58, 'High': 50549.57, 'Low': 50...","{'Open': 50220.01, 'High': 50558.87, 'Low': 50...","{'Open': 50200.0, 'High': 50566.65, 'Low': 500...","{'Open': 50166.25, 'High': 50529.0, 'Low': 500...","{'Open': 50202.